1. Determine the five most common journals and the total articles for each. 
2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [41]:
raw_data = pd.read_csv('input\data_UTF-16LE.csv', encoding='UTF_16_LE')
print(raw_data.shape)
raw_data.head(10)

(2127, 5)


,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


### Exploratory Data Analysis and Cleaning

In [20]:
# Renaming "Cost" column
raw_data.rename({'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost'}, axis=1, inplace=True)
raw_data.rename({'Journal title': 'Journal'}, axis=1, inplace=True)
raw_data.rename({'Article title': 'Article'}, axis=1, inplace=True)
raw_data.rename({'PMID/PMCID': 'PMID'}, axis=1, inplace=True)

# make changes to 'data' rather than raw_data
data = raw_data

# Removing the £ and $ from cost column
data['Cost'].replace(regex=True, inplace=True, to_replace=r'[£$]', value=r'')

# Converting string to float
pd.to_numeric(data['Cost']).head(5)

0       0.00
1    2381.04
2     642.56
3     669.64
4     685.88
Name: Cost, dtype: float64

In [21]:
# Check if there are any null values and in what columns
data.isnull().sum()

PMID/PMCID       199
Publisher          0
Journal title      1
Article title      0
Cost               0
dtype: int64

In [44]:
# Checking to see what all publishers we have
np.unique(data['Publisher'].values)

array(['ACS', 'ACS (Amercian Chemical Society) Publications',
       'ACS Publications', 'AGA Institute', 'AMBSB',
       'AMERICAN CHEMICAL SOCIETY', 'ASBMB', 'ASBMB Cadmus',
       'ASBMB/Cadmus', 'ASBMB/Cenveo Publisher Services', 'ASBMC /CENVEO',
       'ASM', 'ASM (American Society for Microbiology)',
       'American Association of Immunologists',
       'American Chemical Society',
       'American Chemical Society Publications',
       'American College of Chest Physicians',
       'American Physiological Society',
       'American Psychiatric Association',
       'American Psychiatric Publishing',
       'American Psychological Association',
       'American Psychological Association ',
       'American Public Health Association',
       'American Soc for Biochemistry and Molecular Biology',
       'American Society for Biochemistry and Molecular Biolgy',
       'American Society for Biochemistry and Molecular Biology',
       'American Society for Biochemistry and Molecular B

In [43]:
np.unique(data['Journal'].values)

TypeError: '<' not supported between instances of 'str' and 'float'

In [45]:
journal = np.array(data['Journal'])


In [46]:
np.unique(journal[0])

array(['Psychological Medicine'], dtype='<U22')